# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [2]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [3]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")


# Display sample data
vacation_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,57.67,69,40,4.83,scattered clouds
1,1,Vaini,TO,-21.2000,-175.2000,84.36,84,75,9.22,broken clouds
2,2,Baker City,US,44.7749,-117.8344,24.51,58,0,4.61,clear sky
3,3,Ismailia,EG,30.6043,32.2723,61.32,75,11,7.87,few clouds
4,4,Rikitea,PF,-23.1203,-134.9692,74.91,77,7,12.75,clear sky


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [7]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country","Description"],

)
    
    
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [10]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"] == "Fortuna"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Fortuna"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Solano"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Pacific Grove"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Portland"]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [11]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2,
                         vacation_stop3, vacation_end])
itinerary_df



,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
192,192,Fortuna,US,40.5982,-124.1573,49.12,77,100,3.44,light rain
163,163,Solano,US,38.2333,-121.9511,49.80,92,100,10.36,heavy intensity rain
236,236,Pacific Grove,US,36.6177,-121.9166,55.65,92,100,0.00,overcast clouds
264,264,Portland,US,45.5234,-122.6762,40.93,59,20,24.16,few clouds
192,192,Fortuna,US,40.5982,-124.1573,49.12,77,100,3.44,light rain


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [12]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df.copy()

# Display sample data
waypoints_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
192,192,Fortuna,US,40.5982,-124.1573,49.12,77,100,3.44,light rain
163,163,Solano,US,38.2333,-121.9511,49.80,92,100,10.36,heavy intensity rain
236,236,Pacific Grove,US,36.6177,-121.9166,55.65,92,100,0.00,overcast clouds
264,264,Portland,US,45.5234,-122.6762,40.93,59,20,24.16,few clouds
192,192,Fortuna,US,40.5982,-124.1573,49.12,77,100,3.44,light rain


## Use GeoViews to create map that shows the four cities in the itinerary

In [21]:
# Configure the map plot by using the itineraty_df
waypoints_map = waypoints_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["City", "Hotel Name", "Country", "Current Description"]
)




In [20]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Country,Current Description)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [15]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [16]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '12713731393340b5bb0f7f1b5f5c67ec',
 'waypoints': '40.5982,-124.1573|38.2333,-121.9511|36.6177,-121.9166|45.5234,-122.6762|40.5982,-124.1573'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [17]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [22]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["geometry"]["coordinates"]

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [23]:
# Create and empty list to store the longitude of each step
longitude=[]

# Create and empty list to store the latitude of step
latitude=[]

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for l in legs: 
    for x in l:
        longitude.append(x[0])
        latitude.append(x[1])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [25]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame

route_df['Longitude'] = longitude
route_df['Latitude'] = latitude

# Display sample data
route_df.head()


,Longitude,Latitude
0,-124.157303,40.598392
1,-124.157472,40.598393
2,-124.157571,40.598392
3,-124.157571,40.598067
4,-124.157576,40.597985


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [26]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path([route_df])

In [27]:
# Display a composed plot by using the route_map and route_path objects
route_path()


:Path   [Longitude,Latitude]